In [ ]:
# import the library
from pynq import Overlay     # import the overlay
from pynq import allocate    # import for CMA (contingeous memory allocation)
from pynq import DefaultIP   # import the ip connector library for extension
import numpy as np
import os
import subprocess
import re
import dfx4ml.magicSeq as magicSeq  # import the magic sequence library
import dfx4ml.dfxCtrl as dfxCtrl  # import the dfx control library
import dfx4ml.cap     as cap
import dfx4ml.memAlloc as dataAlloc  # import the memory allocation library

PRJ_DIR    = '/home/xilinx/jupyter_notebooks/tanawin/dfx4ml_magicSkipper/'
PRJ_HW_DIR = '/home/xilinx/jupyter_notebooks/tanawin/dfx4ml_magicSkipper/hw/'

DFX_CONFIG_FILE = 'dfxCtrlMeta.txt'

FULL_BS_NAME    = 'system.bin'
PAR_BS_NAME_0   = 'skipper1.bin' ###### dma to magic stream 1
PAR_BS_NAME_1   = 'skipper2.bin' ###### magic stream 1 to magic stream 2


AMT_QUERY       = 1
INPUT_SHAPE     = (4,4,1)  # 4*4* float32 = 64 bytes
# intermediate layer = (4*4*8* float32 = 512 bytes)
# intermediate layer = (4*4*8* float32 = 512 bytes)
OUTPUT_SHAPE    = (4,4,1)  # 4*4* float32 = 64 bytes
AMT_SLOT = 3

In [ ]:
cap.changePLconfigMode("pcap", True)

In [ ]:
#### load the overlay
overlay  = Overlay(PRJ_HW_DIR + FULL_BS_NAME)

In [ ]:
#### get the device
dmaIp      = overlay.dataMovement.axi_dma_0
dfxCtrlIp  = overlay.PRcontroller.dfx_controller_0
magicSeqIp = overlay.magicSeq.MagicSeqTopIntr_0

In [ ]:
#### configure the dfx controller ip to match the address space
dfxCtrlIp.config(PRJ_HW_DIR + DFX_CONFIG_FILE)
print("regIdxSize = ", dfxCtrlIp.BLS_REGID)

In [ ]:
### change reconfigure mode
cap.changePLconfigMode("icap", True)

In [ ]:
dfxCtrlIp.printStatus()

In [ ]:
#### shutdown all system

magicSeqIp.shutdownEngine()
dfxCtrlIp .shutdownEngine()

In [ ]:
# get physical address of dma and dfx controller
dmaPhyAddr     =  overlay.ip_dict['axi_dma_0']['phys_addr']
dfxCtrlPhyAddr =  overlay.ip_dict['dfx_controller_0']['phys_addr']

print("dma physical address: ", hex(dmaPhyAddr))
print("dfx  Ctrl physical address: ", hex(dfxCtrlPhyAddr))

In [ ]:
##### initialize magic seq
print("------ before init magic seq------")
print(magicSeqIp.printDebug())

print("------ init magic sequence METADATA bank 0 -------------------------")
magicSeqIp.setEndCnt(2) ### use the last index
magicSeqIp.setDmaAddr(dmaPhyAddr)
magicSeqIp.setDfxAddr(dfxCtrlPhyAddr)
magicSeqIp.setIntrEna(0)
magicSeqIp.setIntr(1)  # woc  command 1 to set the interrupt to 0
magicSeqIp.setRoundTrip(0)  # set round trip to 0, no need to wait for the dma to finish
inputX = np.arange(INPUT_SHAPE[0], dtype=np.float32)
print("-------------init all data buffer -------------")
buf_input   , buf_input_phya   , buf_input_sz    = dataAlloc.allocDataUint(allocShape= INPUT_SHAPE, allocType= np.float32, inputX = inputX)
buf_out     , buf_out_phy      , buf_out_sz      = dataAlloc.allocDataUint(allocShape= OUTPUT_SHAPE  , allocType= np.float32)

print("------------- init all bank 1 ------------------")
######                      srcPhyAddr    ,        srcSz,  dstPhyAddr,      dstSz,st,pr,loadMask, storeMask, intrMask 
magicSeqIp.setWholeSlot(0, [buf_input_phya, buf_input_sz,           0,          0, 0, 0,  0b0001,    0b0110, 0])
magicSeqIp.setWholeSlot(1, [             0,            0, buf_out_phy, buf_out_sz, 0, 0,  0b0110,    0b0001, 0])

print("------------- after init magic seq------")
print(magicSeqIp.printDebug())

In [ ]:
##### initialize dfx controller
print("------ allocate bit steram CMA for each trigger ------")

######## set trigger 0
d0_ip_buf, d0_addr, d0_size = \
    dfxCtrlIp.allocateBitStreamCMA(PRJ_HW_DIR + PAR_BS_NAME_0)
######## set trigger 1
d1_ip_buf, d1_addr, d1_size = \
    dfxCtrlIp.allocateBitStreamCMA(PRJ_HW_DIR + PAR_BS_NAME_1)

In [ ]:
##### initialize dfx controller2
dfxCtrlIp.setSimpleMetaData(0, d0_addr, d0_size)
dfxCtrlIp.setSimpleMetaData(1, d1_addr, d1_size)

In [ ]:
##### check dfx controller3
dfxCtrlIp.printStatus()
dfxCtrlIp.printSimpleMetaData(0)
dfxCtrlIp.printSimpleMetaData(1)

In [ ]:
dfxCtrlIp.trigger(0)
dfxCtrlIp.restartNoStatus()

In [ ]:
dfxCtrlIp.printStatus()

In [ ]:
##### start dfx controller3
magicSeqIp.clearIntr()
magicSeqIp.startEngine()

In [ ]:
print(magicSeqIp.printDebug())

In [ ]:
np_parRes = np.array(buf_out, dtype=np.float32)
print(np_parRes)